# AZML III Pipeline - hyper parametre et Auto ml
Laurent Cetinsoy


## AutoML

Le machine learning, c'est bien. Le terme buzz du moment c'est AutoML : automatiser la recherche du meilleur modèle ou des meilleurs hyper paramètres pour un dataset donné. Plein d'entreprises proposent des outils pour le faire et microsoft ne déroge pas à la règle : le sdk permet d'en faire.  

Reférence utile :

- https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train
- https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml

Les concepts / classes utiles sont : AutoMLConfig

Charger le Workspace comme d'hab

### Regression

On va entraîner un modèle de régression sur le dataset house. Charger le dataset house.csv avec pandas

Faire un train test split

Créer un objet de la classe AutoMLConfig. Configurer l'experience pour que ça ne dure pas trop longtemps  avec notamment les paramètre experiment_timeout_minute et n_cross_validations

Créer une experience "myautoml_classification"

Lancer votre experience avec le submit sur l'objet de config

Si ça fail, ce n'est pas très grave. Essayez de débug mais ne perdez pas trop de temps. Automl a besoin de bcp de librairies et le faire marcher en local est galère. Passer plutôt en remote ou lancer un notebook sur votre workspace azure

## Option en remote

Pour cela on va devoir utiliser une ressource en remote. 
Récupérer une référence au AmlCompute du notebook précédent si vous l'avez fait.
Sinon en créér un grâce à la méthode provisioning_configuration de la classe AmlCompute

Créer une instance de la classe ComputeTarget

### Upload du dataset

en remote on est obligé d'upload notre dataset sur un DataStore et récupérer une référence via un `Dataset`. En effet on ne peut pas envoyer tel quel un df pandas en mémoire vers la compute target distante.

Uploader et récupérer une référence du fichier houses_automl.csv

Créer une AutoMLConfig qui utilise le dataset uploadé et la compute target à distance

Créer une experience "remoteautoml". Submit l'experience avec l'objet automl config que vous venez de créer

Vérifier que cela a bien marché et récupérer le meilleur modèle

## Hyper drive 

Kezako ? Module d'azure machine learning qui permet de faire de la recherche d'hyper parametres.

Reférences utiles : 
- https://docs.microsoft.com/fr-fr/python/api/azureml-train-core/azureml.train.hyperdrive?view=azure-ml-py
- https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters


Les classes concepts intéressant sont HyperParameterSampling, HyperDriveRun

Quelle différence entre hyperdrive et automl ? C'est pas la même chose ? Pourquoi deux choses différentes ? 

automl teste plusieurs modèles, hyperdrive teste plusieurs hyperparamètres d'un même modèle

A quoi sert la classe HyperParameterSampling ? 


Permet de spécifier comment on "chercher" des hyperparametres (cf https://docs.microsoft.com/fr-fr/python/api/azureml-train-core/azureml.train.hyperdrive.hyperparametersampling?view=azure-ml-py)

Quelles sont les principales classes filles ? 

"Cette classe encapsule l’espace hyperparamétrique, la méthode d’échantillonnage et les propriétés supplémentaires pour les classes d’échantillonnage dérivées : BayesianParameterSampling , GridParameterSampling et RandomParameterSampling ."

Quelles semblent-être les méthodes les plus importantes ? 

A quoi sert la classe HyperDriveRun ? 

Classe fille de la classe run, c'est une classe particulière qui permet de récupérer les résultats de la recherche d'hyperparamètres.

On va faire la recherche d'hyperparamètre sur notre modèle de régression de tout à l'heure (house).
Créer un dictionnaire avec les noms des paramètres que vous voulez recherche (par exemple le learning rate ou le taux de régularisation), cf https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#define-search-space

Les faire commencer par un --. Par exemple --n_epochs et pas n_epochs. Cela servira pour passer les arguments à notre script d'entraînement

Instancier un objet de la classe RandomParameterSampling 

Spécifier la métrique que vous voulez optimiser

On va maximiser le R2 qu'on déclarera dans les paramètres de HyperDriveConfig

Choisir un critère d'arrêt de la recherche d'hyper paramètre 

Editer le fichier d'entraînement train_house.py pour qu'il soit compatible avec l'entraînement. Il faut utiliser le module argpass pour récupérer les paramètres dont vous voulez faire la recherche. Cf https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters

! Editer le fichier !

Créer un estimateur SkLearn

Créer une configuration HyperdriveConfiguration

Soumettre l'experience localement

Et récupérer le meilleur set d'hyperparamètres